In [1]:
import os
import wget
import healpy as hp
import numpy as np
from tqdm import tqdm
from astropy.io import fits as pyfits
import random
import pymp
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from threadpoolctl import threadpool_limits
import importlib
import sys
import multiprocessing as mp
from math import sqrt
from math import radians
import gc
import subprocess
gc.disable()
#######################################################################################################
from cmbaberdopp import * # cmbboost its a module created w/ the functions needed for this work          #
                       # remember to change the path of maps on dipole_doppler_load_low_res function  #
#######################################################################################################
def c_nthreads_limit(n): # Function to change number of threads used on imported C/C++ functions
    threadpool_limits(limits=n, user_api='blas')
    threadpool_limits(limits=n, user_api='openmp')

In [ ]:
# Download Planck final SMICA and NILC maps to get realistic beam function

wget.download('http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CMB_IQU-smica_2048_R3.00_full.fits')
wget.download('http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CMB_IQU-nilc_2048_R3.00_full.fits')

# Debeamed

In [ ]:
#### Importing effective beam and masks files####

lmax_var = 2048
pw = hp.pixwin(nside=2048,lmax=lmax_var)

clfile_TT='cl_TT_planck_2019.fits'
cl_TT = hp.read_cl(clfile_TT)
clfile_EE='cl_EE_planck_2019.fits'
cl_EE = hp.read_cl(clfile_EE)
clfile_TE='cl_TE_planck_2019.fits'
cl_TE = hp.read_cl(clfile_TE)


nilcgb = pyfits.open('COM_CMB_IQU-nilc_2048_R3.00_full.fits')[2].data['INT_BEAM'][0:lmax_var]
nilcgb[0]=1
nilcgb[1]=1
#hp.write_cl('nilc_T_effective_beam.fits',nilcgb)
nilcgb_P = pyfits.open('COM_CMB_IQU-nilc_2048_R3.00_full.fits')[2].data['POL_BEAM'][0:lmax_var]
nilcgb_P[0]=1
nilcgb_P[1]=1
#hp.write_cl('nilc_P_effective_beam.fits',nilcgb_P)
smicagb = pyfits.open('COM_CMB_IQU-smica_2048_R3.00_full.fits')[2].data['INT_BEAM'][0:lmax_var]
smicagb[0]=1
smicagb[1]=1
#hp.write_cl('smica_T_effective_beam.fits',smicagb)
smicagb_P = pyfits.open('COM_CMB_IQU-smica_2048_R3.00_full.fits')[2].data['POL_BEAM'][0:lmax_var]
smicagb_P[0]=1
smicagb_P[1]=1
#hp.write_cl('smica_P_effective_beam.fits',smicagb_P)

masksymm_temp = hp.read_map('temp_symmmask_pr3.fits',verbose=False)
masksymm_pol = hp.read_map('pol_symmmask_pr3.fits',verbose=False)


In [ ]:
#Debeamed noise
nilc_T_nl = hp.read_cl('nl_nilc_T_symmmasked.fits')
smica_T_nl = hp.read_cl('nl_smica_T_symmmasked.fits')
nilc_E_nl = hp.read_cl('nl_nilc_E_symmmasked.fits')
smica_E_nl = hp.read_cl('nl_smica_E_symmmasked.fits')

In [3]:
pw = hp.pixwin(nside=2048,lmax=2048)

In [ ]:
c_nthreads_limit(2)

##### Importing Planck OBS maps #####
nilc_I_obs_map, nilc_Q_obs_map, nilc_U_obs_map = hp.read_map('COM_CMB_IQU-nilc_2048_R3.00_full.fits',field=(0,1,2))
nilc_T_obs_alm, nilc_E_obs_alm, nilc_B_obs_alm = hp.map2alm((nilc_I_obs_map,nilc_Q_obs_map, nilc_U_obs_map))
nilc_T_obs_debeamed_alm = hp.almxfl(nilc_T_obs_alm,1/(nilcgb[0:lmax_var]))
nilc_E_obs_debeamed_alm = hp.almxfl(nilc_E_obs_alm,1/(nilcgb_P[0:lmax_var]))
nilc_B_obs_debeamed_alm = hp.almxfl(nilc_B_obs_alm,1/(nilcgb_P[0:lmax_var]))

hp.write_alm('planck_obs_nilc_T_obs_debeamed_alm.fits',nilc_T_obs_debeamed_alm,overwrite=True)
hp.write_alm('planck_obs_nilc_E_obs_debeamed_alm.fits',nilc_E_obs_debeamed_alm,overwrite=True)
hp.write_alm('planck_obs_nilc_B_obs_debeamed_alm.fits',nilc_B_obs_debeamed_alm,overwrite=True)

smica_I_obs_map, smica_Q_obs_map, smica_U_obs_map = hp.read_map('COM_CMB_IQU-smica_2048_R3.00_full.fits',field=(0,1,2))
smica_T_obs_alm, smica_E_obs_alm, smica_B_obs_alm = hp.map2alm((smica_I_obs_map,smica_Q_obs_map, smica_U_obs_map))
smica_T_obs_debeamed_alm = hp.almxfl(smica_T_obs_alm,1/(smicagb[0:lmax_var]))
smica_E_obs_debeamed_alm = hp.almxfl(smica_E_obs_alm,1/(smicagb_P[0:lmax_var]))
smica_B_obs_debeamed_alm = hp.almxfl(smica_B_obs_alm,1/(smicagb_P[0:lmax_var]))

hp.write_alm('planck_obs_smica_T_obs_debeamed_alm.fits',smica_T_obs_debeamed_alm,overwrite=True)
hp.write_alm('planck_obs_smica_E_obs_debeamed_alm.fits',smica_E_obs_debeamed_alm,overwrite=True)
hp.write_alm('planck_obs_smica_B_obs_debeamed_alm.fits',smica_B_obs_debeamed_alm,overwrite=True)

nilc_I_obs_debeamed_map, nilc_Q_obs_debeamed_map, nilc_U_obs_debeamed_map = hp.alm2map((nilc_T_obs_debeamed_alm,nilc_E_obs_debeamed_alm,nilc_B_obs_debeamed_alm),nside=2048)
smica_I_obs_debeamed_map, smica_Q_obs_debeamed_map, smica_U_obs_debeamed_map = hp.alm2map((smica_T_obs_debeamed_alm,smica_E_obs_debeamed_alm,smica_B_obs_debeamed_alm),nside=2048)

nilc_I_obs_symmasked_map = nilc_I_obs_debeamed_map*masksymm_temp
nilc_Q_obs_symmasked_map = nilc_Q_obs_debeamed_map*masksymm_pol
nilc_U_obs_symmasked_map = nilc_U_obs_debeamed_map*masksymm_pol
smica_I_obs_symmasked_map = smica_I_obs_debeamed_map*masksymm_temp
smica_Q_obs_symmasked_map = smica_Q_obs_debeamed_map*masksymm_pol
smica_U_obs_symmasked_map = smica_U_obs_debeamed_map*masksymm_pol

In [ ]:
nilc_alm_T_debeamed_symmasked,nilc_alm_E_debeamed_symmasked,nilc_alm_B_debeamed_symmasked = hp.map2alm((nilc_I_obs_symmasked_map,nilc_Q_obs_symmasked_map,nilc_U_obs_symmasked_map),lmax=2048)
smica_alm_T_debeamed_symmasked,smica_alm_E_debeamed_symmasked,smica_alm_B_debeamed_symmasked = hp.map2alm((smica_I_obs_symmasked_map,smica_Q_obs_symmasked_map,smica_U_obs_symmasked_map),lmax=2048)

hp.write_alm('planck_obs_nilc_debeamed_symmasked_alm_T.fits',nilc_alm_T_debeamed_symmasked)
hp.write_alm('planck_obs_smica_debeamed_symmasked_alm_T.fits',smica_alm_T_debeamed_symmasked)
hp.write_alm('planck_obs_nilc_debeamed_symmasked_alm_E.fits',nilc_alm_E_debeamed_symmasked)
hp.write_alm('planck_obs_smica_debeamed_symmasked_alm_E.fits',smica_alm_E_debeamed_symmasked)

In [ ]:
clTTnilcOBS, clEEnilcOBS, clBBnilcOBS, clTEnilcOBS, clEBnilcOBS, clTBnilcOBS  = hp.anafast((nilc_I_obs_symmasked_map,nilc_Q_obs_symmasked_map,nilc_U_obs_symmasked_map),lmax=2048,iter=3)
clTTsmicaOBS, clEEsmicaOBS, clBBsmicaOBS, clTEsmicaOBS, clEBsmicaOBS, clTBsmicaOBS  = hp.anafast((nilc_I_obs_symmasked_map,nilc_Q_obs_symmasked_map,nilc_U_obs_symmasked_map),lmax=2048,iter=3)

hp.write_cl('planck_obs_nilc_debeamed_symmasked_cl_TT.fits',clTTnilc)
hp.write_cl('planck_obs_smica_debeamed_symmasked_cl_TT.fits',clTTsmica)
hp.write_cl('planck_obs_nilc_debeamed_symmasked_cl_EE.fits',clEEnilc)
hp.write_cl('planck_obs_smica_debeamed_symmasked_cl_EE.fits',clEEsmica)

In [ ]:
lmax = 2048

mastermatrixfileTT='master_matrix_TT_pr3.csv'
mastermatrixTT=np.genfromtxt(mastermatrixfileTT, delimiter=',')
mastermatrixTT[:,0:2]=0
mastermatrizsizeTT=len(mastermatrixTT)
cl_TT=cl_TT[:mastermatrizsizeTT]
cl_TT=np.dot(mastermatrixTT,cl_TT)[0:lmax+2]

mastermatrixfileEE='master_matrix_EE_pr3.csv'
mastermatrixEE=np.genfromtxt(mastermatrixfileEE, delimiter=',')
mastermatrixEE[:,0:2]=0
mastermatrizsizeEE=len(mastermatrixEE)
cl_EE=cl_EE[:mastermatrizsizeEE]
cl_EE=np.dot(mastermatrixEE,cl_EE)[0:lmax+2]

sims_nilc_cl_TT = cl_TT[:lmax+1]+nilc_T_nl[:lmax+1]
sims_smica_cl_TT = cl_TT[:lmax+1]+smica_T_nl[:lmax+1]
sims_nilc_cl_EE = cl_EE[:lmax+1]+nilc_E_nl[:lmax+1]
sims_smica_cl_EE = cl_EE[:lmax+1]+smica_E_nl[:lmax+1]

In [ ]:
bin_size = 10
nilc_TT_beta_corr_bin_factors = np.array([(np.sum((sims_nilc_cl_TT[bin*bin_size:(bin+1)*bin_size]/clTTnilcOBS[bin*bin_size:(bin+1)*bin_size])*(2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))/np.sum((2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))) for bin in range(200)])
smica_TT_beta_corr_bin_factors = np.array([(np.sum((sims_smica_cl_TT[bin*bin_size:(bin+1)*bin_size]/clTTsmicaOBS[bin*bin_size:(bin+1)*bin_size])*(2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))/np.sum((2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))) for bin in range(200)])
nilc_EE_beta_corr_bin_factors = np.array([(np.sum((sims_nilc_cl_EE[bin*bin_size:(bin+1)*bin_size]/clEEnilcOBS[bin*bin_size:(bin+1)*bin_size])*(2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))/np.sum((2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))) for bin in range(200)])
smica_EE_beta_corr_bin_factors = np.array([(np.sum((sims_smica_cl_EE[bin*bin_size:(bin+1)*bin_size]/clEEsmicaOBS[bin*bin_size:(bin+1)*bin_size])*(2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))/np.sum((2*np.arange(bin*bin_size,(bin+1)*bin_size)+1))) for bin in range(200)])

np.savetxt('nilc_TT_beta_corr_bin_factors.txt',nilc_TT_beta_corr_bin_factors)
np.savetxt('smica_TT_beta_corr_bin_factors.txt',smica_TT_beta_corr_bin_factors)
np.savetxt('nilc_EE_beta_corr_bin_factors.txt',nilc_EE_beta_corr_bin_factors)
np.savetxt('smica_EE_beta_corr_bin_factors.txt',smica_EE_beta_corr_bin_factors)

In [ ]:
#### Estimating NILC TT symmmasked Beta ####

htheofast(sims_nilc_cl_TT,binsize=10,plot_std=True,threads=8)
nilc_alm_T_debeamed_symmasked = reorder_idxpy2pix(nilc_alm_T_debeamed_symmasked,threads=8)
betafast(nilc_alm_T_debeamed_symmasked,binsize=10,threads=12,sufixname='nilc_T_obs_symmmasked_',verbose=False,export_results=True);


In [ ]:
#### Estimating SMICA TT symmmasked Beta ####

htheofast(sims_smica_cl_TT,binsize=10,plot_std=True,threads=8)
smica_alm_T_debeamed_symmasked = reorder_idxpy2pix(smica_alm_T_debeamed_symmasked,threads=8)
betafast(smica_alm_T_debeamed_symmasked,binsize=10,threads=12,sufixname='smica_T_obs_symmmasked_',verbose=False,export_results=True);


In [ ]:
#### Estimating NILC EE symmmasked Beta ####

htheofast(sims_nilc_cl_EE,binsize=10,plot_std=True,threads=8)
nilc_alm_E_debeamed_symmasked = reorder_idxpy2pix(nilc_alm_E_debeamed_symmasked,threads=8)
betafast(nilc_alm_E_debeamed_symmasked,binsize=10,threads=12,sufixname='nilc_E_obs_symmmasked_',verbose=False,export_results=True);


In [ ]:
#### Estimating SMICA EE symmmasked Beta ####

htheofast(sims_smica_cl_EE,binsize=10,plot_std=True,threads=8)
smica_alm_E_debeamed_symmasked = reorder_idxpy2pix(smica_alm_E_debeamed_symmasked,threads=8)
betafast(smica_alm_E_debeamed_symmasked,binsize=10,threads=12,sufixname='smica_E_obs_symmmasked_',verbose=False,export_results=True);


In [ ]:
plt.plot(sims_nilc_cl_TT*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='b')
plt.plot(nilc_T_nl*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='b', linestyle='--')
plt.plot(sims_smica_cl_TT*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='r')
plt.plot(smica_T_nl*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='r', linestyle='--')
plt.grid(color='k', linestyle='-', linewidth=0.5)
plt.show()
plt.plot(sims_nilc_cl_EE*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='b')
plt.plot(nilc_E_nl*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='b', linestyle='--')
plt.plot(sims_smica_cl_EE*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='r')
plt.plot(smica_E_nl*((np.arange(0,sims_nilc_cl_TT.size)*np.arange(0,sims_nilc_cl_TT.size)+1))/(2*np.pi),color='r', linestyle='--')
plt.grid(color='k', linestyle='-', linewidth=0.5)
plt.show()


# Deboosted 
You have to generate the Dipole Distortion maps before call the "dipole_doppler_".
Maybe you will need to shutdown and restart the kernel before run


In [ ]:
###############################
####### DEBOOSTED ALMS ########
########### SMICA #############
###############################

smica_T_obs_debeamed_alm = hp.read_alm('planck_obs_smica_T_obs_debeamed_alm3.fits')
alm=smica_T_obs_debeamed_alm
dd_maps_dir='/dd_maps/'
dipole_doppler_load_low_res(alm=alm,pipeline='smica',almfile=False,lmax=2048,overbin=10,threads=4,verbose=True,dd_dir=dd_maps_dir)

start = time.time()
smica_dd_alm = dipole_doppler_low_res_removal(smica_T_obs_debeamed_alm,threads=4,processes=1)
hp.write_alm('planck_obs_smica_dd_deboosted_debeamed4.fits',smica_dd_alm,overwrite=True)
end = time.time()

In [ ]:
###############################
####### DEBOOSTED ALMS ########
########### NILC ##############
###############################

nilc_T_obs_debeamed_alm = hp.read_alm('planck_obs_nilc_T_obs_debeamed_alm3.fits')
alm=nilc_T_obs_debeamed_alm
dd_maps_dir='/dd_maps/'
dipole_doppler_load_low_res(alm=alm,pipeline='nilc',almfile=False,lmax=2048,overbin=10,threads=4,verbose=True,dd_dir=dd_maps_dir)

start = time.time()
nilc_dd_alm = dipole_doppler_low_res_removal(nilc_T_obs_debeamed_alm,threads=4,processes=4)
hp.write_alm('planck_obs_nilc_dd_deboosted_debeamed4.fits',nilc_dd_alm,overwrite=True)
end = time.time()


In [ ]:
masksymm_temp = hp.read_map('temp_symmmask_pr3.fits',verbose=False)
lmax_var = 2048

#### Masking ####
nilc_obs_debeamed_alm = hp.read_alm('planck_obs_nilc_dd_deboosted_debeamed4.fits')
smica_obs_debeamed_alm = hp.read_alm('planck_obs_smica_dd_deboosted_debeamed4.fits')

nilc_obs_debeamed_map = hp.alm2map(nilc_obs_debeamed_alm,nside=2048,verbose=False)
nilc_obs_debeamed_symmmasked_map = nilc_obs_debeamed_map*masksymm_temp

smica_obs_debeamed_map = hp.alm2map(smica_obs_debeamed_alm,nside=2048,verbose=False)
smica_obs_debeamed_symmmasked_map = smica_obs_debeamed_map*masksymm_temp

#### Final alms ####

nilc_obs_debeamed_symmmasked_alm = hp.map2alm(nilc_obs_debeamed_symmmasked_map,lmax=lmax_var,iter=3)
smica_obs_debeamed_symmmasked_alm = hp.map2alm(smica_obs_debeamed_symmmasked_map,lmax=lmax_var,iter=3)

In [ ]:
#### Estimating SMICA symmmasked Beta ####
clfile='cl_TT_planck_2019.fits'
cl = hp.read_cl(clfile)

htheofast(cl,mastermatrixfile='master_matrix_TT_pr3.csv',nlfile='nl_smica_T_symmmasked.fits',binsize=10,plot_std=True,threads=8)

smica_obs_debeamed_symmmasked_alm = reorder_idxpy2pix(smica_obs_debeamed_symmmasked_alm,threads=8)

betafast(smica_obs_debeamed_symmmasked_alm,binsize=10,threads=12,sufixname='smica_obs_dd_deboosted_symmmasked_v2_',verbose=False,export_results=True);


In [ ]:
#### Estimating NILC symmmasked Beta ####
clfile='cl_TT_planck_2019.fits'
cl = hp.read_cl(clfile)

htheofast(cl,mastermatrixfile='master_matrix_TT_pr3.csv',nlfile='nl_nilc_T_symmmasked.fits',binsize=10,plot_std=True,threads=8)

nilc_obs_debeamed_symmmasked_alm = reorder_idxpy2pix(nilc_obs_debeamed_symmmasked_alm,threads=8)

betafast(nilc_obs_debeamed_symmmasked_alm,binsize=10,threads=12,sufixname='nilc_obs_dd_deboosted_symmmasked_v2_',verbose=False,export_results=True);


In [ ]:
alm_E = hp.read_alm('planck_obs_smica_E_obs_debeamed_alm3.fits')
dipole_doppler_load_low_res_pol(alm=alm_E,pipeline='smica',almfile=False,lmax=1450,overbin=10,threads=16,dd_dir="/dd_maps/",verbose=True)

nthreads = 4
c_nthreads_limit(nthreads)
nprocesses = 3

alm_E_dd_removed = dipole_doppler_low_res_pol_removal(alm_E,lmax=1450,threads=nthreads,processes=nprocesses)
hp.write_alm('smica_EE_v2_deDopplered_.fits',alm_E_dd_removed,overwrite=True)


In [ ]:
alm_E = hp.read_alm('planck_obs_nilc_E_obs_debeamed_alm3.fits')
dipole_doppler_load_low_res(alm=alm_E,pipeline='nilc',almfile=False,lmax=1450,overbin=10,threads=16,dd_dir="/dd_maps/",verbose=True)

nthreads = 4
c_nthreads_limit(nthreads)
nprocesses = 3

alm_E_dd_removed = dipole_doppler_low_res_pol_removal(alm_E,lmax=1450,threads=nthreads,processes=nprocesses)
hp.write_alm('nilc_EE_v2_deDopplered_.fits',alm_E_dd_removed,overwrite=True)


In [ ]:
masksymm_pol = hp.read_map('pol_symmmask_pr3.fits',verbose=False)

alm_E_deDoppler = hp.read_alm('smica_EE_v2_deDopplered_.fits')
alm_T = np.zeros(alm_E_deDoppler.size,dtype=np.complex128)
alm_B = np.zeros(alm_E_deDoppler.size,dtype=np.complex128)
map_I, map_Q, map_U = hp.alm2map([alm_T,alm_E_deDoppler,alm_B],nside=2048)
map_I = map_I*masksymm_pol
map_Q = map_Q*masksymm_pol
map_U = map_U*masksymm_pol
alm_T, alm_E_deDoppler, alm_B = hp.map2alm([map_I,map_Q,map_U],lmax=1450)

clfile='cl_EE_planck_2019.fits'
cl = hp.read_cl(clfile)

htheofast(cl,lmax=1450,mastermatrixfile='master_matrix_EE_pr3.csv',nlfile='nl_smica_E_symmmasked.fits',binsize=10,threads=8)

smica_debeamed_masked_alm_EE = reorder_idxpy2pix(alm_E_deDoppler,threads=8)

betafast(smica_debeamed_masked_alm_EE,lmax=1442,binsize=10,threads=12,sufixname='new_smica_EE_deDopplered_obs_symmmasked_',verbose=False,export_results=True);


In [ ]:
masksymm = hp.read_map('/home/pedro/hp_nvme/cmb_boost_legacy_v2/Pol_mask/pol_symmmask.fits')

alm_E_deDoppler = hp.read_alm('nilc_EE_v2_deDopplered_.fits')
alm_T = np.zeros(alm_E_deDoppler.size,dtype=np.complex128)
alm_B = np.zeros(alm_E_deDoppler.size,dtype=np.complex128)
map_I, map_Q, map_U = hp.alm2map([alm_T,alm_E_deDoppler,alm_B],nside=2048)
map_I = map_I*masksymm
map_Q = map_Q*masksymm
map_U = map_U*masksymm
alm_T, alm_E_deDoppler, alm_B = hp.map2alm([map_I,map_Q,map_U],lmax=1450)

clfile='cl_EE_planck_2019.fits'
cl = hp.read_cl(clfile)

htheofast(cl,lmax=1450,mastermatrixfile='master_matrix_TT_pr3.csv',nlfile='nl_nilc_E_symmmasked.fits',binsize=10,threads=8)

smica_debeamed_masked_alm_EE = reorder_idxpy2pix(alm_E_deDoppler,threads=8)

betafast(smica_debeamed_masked_alm_EE,lmax=1442,binsize=10,threads=12,sufixname='new_nilc_EE_deDopplered_obs_symmmasked_',verbose=False,export_results=True);
